In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import random

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [3]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# ✅ CIFAR-10 Dataset (change `root` if needed for Kaggle)
train_datasets = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_datasets = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_datasets, batch_size=256, shuffle=True)
test_loader = DataLoader(test_datasets, batch_size=256, shuffle=False)

100%|██████████| 170M/170M [00:03<00:00, 48.6MB/s]


In [4]:
def get_resnet18_model():
    model = models.resnet18(weights=None)  # Not using pretrained for CIFAR-10
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()  # Remove the aggressive downsampling
    model.fc = nn.Linear(model.fc.in_features, 10)  # CIFAR-10 has 10 classes
    return model.to(device)

model = get_resnet18_model()

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%")


Epoch [1/10], Loss: 1.4391, Accuracy: 47.23%
Epoch [2/10], Loss: 1.0324, Accuracy: 63.08%
Epoch [3/10], Loss: 0.8551, Accuracy: 69.61%
Epoch [4/10], Loss: 0.7330, Accuracy: 74.21%
Epoch [5/10], Loss: 0.6409, Accuracy: 77.61%
Epoch [6/10], Loss: 0.5845, Accuracy: 79.65%
Epoch [7/10], Loss: 0.5367, Accuracy: 81.20%
Epoch [8/10], Loss: 0.5035, Accuracy: 82.34%
Epoch [9/10], Loss: 0.4713, Accuracy: 83.50%
Epoch [10/10], Loss: 0.4458, Accuracy: 84.53%


In [7]:
def predict_from_index(index):
    image, label = test_datasets[index]
    image_input = image.unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(image_input)
        _, predicted = torch.max(output, 1)

    classes = train_datasets.classes
    predicted_class = classes[predicted.item()]
    true_class = classes[label]

    # Denormalize for visualization
    inv_normalize = transforms.Normalize(
        mean=[-m/s for m, s in zip(mean, std)],
        std=[1/s for s in std]
    )
    image_show = inv_normalize(image).permute(1, 2, 0).cpu().numpy()
    image_show = np.clip(image_show, 0, 1)  # Avoid values outside [0, 1]

    plt.imshow(image_show)
    plt.title(f"Predicted: {predicted_class} | Actual: {true_class}")
    plt.axis("off")
    plt.show()